In [1]:
from scripts.information_extractor_lib import InformationExtractor, preprocess_raw_text, is_sentence
from scripts.config_file import configs, generation_config
import pandas as pd
from full_workflow_config import fw_config
import re

## Testing Data - muss geladen wurden siehe get_data_from_ddbapi

In [3]:
n = 10 # number of rows we want to process
df = pd.read_pickle("ddbapi_köln.pkl")
df = df.tail(n).reset_index(drop=True)
df

,page_id,pagenumber,paper_title,provider_ddb_id,provider,zdb_id,publication_date,place_of_distribution,language,thumbnail,pagefulltext,pagename,preview_reference,plainpagefulltext
0,ZXFS2473UNWC3CHAEZ72NJHVO25D2MS4-ALTO10099161_...,4,Rheinischer Merkur : Kölnische Landeszeitung,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,3048191-0,1903-06-05 12:00:00,[Köln],[ger],0b57ac2c-75de-401d-aed7-7a5bc76498c3,/data/altos/ZX/FS/ZXFS2473UNWC3CHAEZ72NJHVO25D...,ALTO10099161_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,M EN K Nebenverdienst erwirbt sich jeder durch...
1,ZXU6KYKLG5L4S5DGG3MPQASQD5V2ML3P-ALTO7642365_D...,12,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1901-01-03 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],f3c70303-17a7-4f7b-bf0d-5b665fdd1e70,/data/altos/ZX/U6/ZXU6KYKLG5L4S5DGG3MPQASQD5V2...,ALTO7642365_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,Lesegesellschaft . Wir erfüllen hiermit die tr...
2,ZY2Z6ZIEIEB3Y3TYZ5XPUWUGP4A7T7XQ-ALTO8681078_D...,16,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1912-07-13 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],4e6142d2-ae45-4c14-a852-9c755a5a257d,/data/altos/ZY/2Z/ZY2Z6ZIEIEB3Y3TYZ5XPUWUGP4A7...,ALTO8681078_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,T Ee PII PRVVEL DISUr ! Umbau vollendet : : Zi...
3,ZY5B654V3GOD5BG53AUUT7RZWZIAGER6-ALTO7678198_D...,8,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1901-09-27 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],96c880ef-1737-4bfd-8fbe-d2e4f0330996,/data/altos/ZY/5B/ZY5B654V3GOD5BG53AUUT7RZWZIA...,ALTO7678198_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"Haasenstein & amp ; Vogler , . - G . Iohestr ...."
4,ZYG2PAXTU45PLUKG5N2QK3CZGZK7BXRC-ALTO9729468_D...,8,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1924-11-23 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],cc4d5aea-fc1e-4cf3-8038-2e7f1623b756,/data/altos/ZY/G2/ZYG2PAXTU45PLUKG5N2QK3CZGZK7...,ALTO9729468_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,Rheinisches Werk auf dem Lande sucht an Vollju...
5,ZYVLIQOXXPQPCF6IA25NUDPGDKTOMSTY-ALTO8702353_D...,16,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1912-09-06 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],bd2f7321-723f-4ef9-9ed7-d1790f9a5818,/data/altos/ZY/VL/ZYVLIQOXXPQPCF6IA25NUDPGDKTO...,ALTO8702353_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,83 Handelsregister - Eintragungen von Amtsgeri...
6,ZZ4LWGXO7L2TEVDZ5F42M46W32X4MX2Q-ALTO8250057_D...,16,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1910-03-04 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],5fc027f8-3575-4e7f-a92e-2f39a129d911,/data/altos/ZZ/4L/ZZ4LWGXO7L2TEVDZ5F42M46W32X4...,ALTO8250057_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"OrSTUT Harmoniums VerFeind , VerLir ! 99 % all..."
7,ZZCJ4QJQXHRQB5RWX2Y7IFC5SXLGD3VV-ALTO7666880_D...,9,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1902-05-28 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],e1f5220c-215b-4cfa-a186-b379ee8a76d2,/data/altos/ZZ/CJ/ZZCJ4QJQXHRQB5RWX2Y7IFC5SXLG...,ALTO7666880_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"Mittwoch , 28 . Mai . A in 8lar : Ugemeinen T ..."
8,ZZKDOIO2RRR54SAIKIBQAHEJNO3QBPWL-ALTO10103160_...,4,Rheinischer Merkur : Kölnische Landeszeitung,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,3048191-0,1906-01-24 12:00

In [4]:
df["org_text"] = df.plainpagefulltext.copy()
df["plainpagefulltext"] = df.plainpagefulltext.apply(lambda s: preprocess_raw_text(s))
df.head(10)

,page_id,pagenumber,paper_title,provider_ddb_id,provider,zdb_id,publication_date,place_of_distribution,language,thumbnail,pagefulltext,pagename,preview_reference,plainpagefulltext,org_text
0,ZXFS2473UNWC3CHAEZ72NJHVO25D2MS4-ALTO10099161_...,4,Rheinischer Merkur : Kölnische Landeszeitung,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,3048191-0,1903-06-05 12:00:00,[Köln],[ger],0b57ac2c-75de-401d-aed7-7a5bc76498c3,/data/altos/ZX/FS/ZXFS2473UNWC3CHAEZ72NJHVO25D...,ALTO10099161_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,M EN K Nebenverdienst erwirbt sich jeder durch...,M EN K Nebenverdienst erwirbt sich jeder durch...
1,ZXU6KYKLG5L4S5DGG3MPQASQD5V2ML3P-ALTO7642365_D...,12,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1901-01-03 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],f3c70303-17a7-4f7b-bf0d-5b665fdd1e70,/data/altos/ZX/U6/ZXU6KYKLG5L4S5DGG3MPQASQD5V2...,ALTO7642365_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,Lesegesellschaft . Wir erfüllen hiermit die tr...,Lesegesellschaft . Wir erfüllen hiermit die tr...
2,ZY2Z6ZIEIEB3Y3TYZ5XPUWUGP4A7T7XQ-ALTO8681078_D...,16,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1912-07-13 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],4e6142d2-ae45-4c14-a852-9c755a5a257d,/data/altos/ZY/2Z/ZY2Z6ZIEIEB3Y3TYZ5XPUWUGP4A7...,ALTO8681078_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,T Ee PII PRVVEL DISUr Umbau vollendet Zimmer m...,T Ee PII PRVVEL DISUr ! Umbau vollendet : : Zi...
3,ZY5B654V3GOD5BG53AUUT7RZWZIAGER6-ALTO7678198_D...,8,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1901-09-27 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],96c880ef-1737-4bfd-8fbe-d2e4f0330996,/data/altos/ZY/5B/ZY5B654V3GOD5BG53AUUT7RZWZIA...,ALTO7678198_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"Haasenstein amp Vogler , . G . Iohestr . 148 1...","Haasenstein & amp ; Vogler , . - G . Iohestr ...."
4,ZYG2PAXTU45PLUKG5N2QK3CZGZK7BXRC-ALTO9729468_D...,8,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1924-11-23 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],cc4d5aea-fc1e-4cf3-8038-2e7f1623b756,/data/altos/ZY/G2/ZYG2PAXTU45PLUKG5N2QK3CZGZK7...,ALTO9729468_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,Rheinisches Werk auf dem Lande sucht an Vollju...,Rheinisches Werk auf dem Lande sucht an Vollju...
5,ZYVLIQOXXPQPCF6IA25NUDPGDKTOMSTY-ALTO8702353_D...,16,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1912-09-06 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],bd2f7321-723f-4ef9-9ed7-d1790f9a5818,/data/altos/ZY/VL/ZYVLIQOXXPQPCF6IA25NUDPGDKTO...,ALTO8702353_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,83 Handelsregister Eintragungen von Amtsgerich...,83 Handelsregister - Eintragungen von Amtsgeri...
6,ZZ4LWGXO7L2TEVDZ5F42M46W32X4MX2Q-ALTO8250057_D...,16,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1910-03-04 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],5fc027f8-3575-4e7f-a92e-2f39a129d911,/data/altos/ZZ/4L/ZZ4LWGXO7L2TEVDZ5F42M46W32X4...,ALTO8250057_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"OrSTUT Harmoniums VerFeind , VerLir 99 aller a...","OrSTUT Harmoniums VerFeind , VerLir ! 99 % all..."
7,ZZCJ4QJQXHRQB5RWX2Y7IFC5SXLGD3VV-ALTO7666880_D...,9,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1902-05-28 12:00:00,"[Köln, Kleve (Kreis Kleve), Jülich]",[ger],e1f5220c-215b-4cfa-a186-b379ee8a76d2,/data/altos/ZZ/CJ/ZZCJ

In [5]:
class FullWorkflow:
    def __init__(self, ids, texts):
        
        self.text_extractor = InformationExtractor(df=None, page_id_colname=None, text_colname=None, output_filename=None)
        self.text_corrector = InformationExtractor(df=None, page_id_colname=None, text_colname=None, output_filename=None)
        self.word_completor = InformationExtractor(df=None, page_id_colname=None, text_colname=None, output_filename=None)
        self.id:list = ids
        self.text:list = texts
        self.text_extraction_result = None
        self.data_corrected = None
    
    def make_input(self):
        if isinstance(self.id,list) and  isinstance(self.text,list) :
            
            return pd.DataFrame(
                data={
                    'id':self.id,
                    'text':self.text
                }
            )
        else:
            return pd.DataFrame(
                data={
                    'id':[self.id],
                    'text':[self.text]
                }
            )

    def reduce_df(self, df):
        df = df[df.text.apply(lambda s: len(s) > 50)].reset_index(drop=True)
        df = df[df.text.apply(lambda s: s != [])].reset_index(drop=True)
        df = df[df.text.apply(lambda s: "Error" not in s)].reset_index(drop=True)
        df = df.reset_index(drop=True)
        return df


    def text_extraction(self):
        self.text_extractor.sleeping_time = 1
        self.text_extractor.save_one_result_per_row = True

        input_df = self.make_input()
        self.text_extractor.df, self.text_extractor.page_id_colname, self.text_extractor.text_colname, self.text_extractor.output_filename = input_df, "id", "text", None
        n_max = len(input_df)
        self.text_extraction_result = self.text_extractor.extract_data_loop(n_max)
        self.text_extraction_result = self.reduce_df(self.text_extraction_result)



    def text_correction(self):

        self.text_corrector.df, self.text_corrector.page_id_colname, self.text_corrector.text_colname, self.text_corrector.output_filename = self.text_extraction_result, "page_id", "text", None
        n_max= len(self.text_extraction_result)
        self.text_corrector.prompt = fw_config["CORRECTION_PROMPT"]
        self.text_corrector.model_name = fw_config["model_name"]
        self.data_corrected = self.text_corrector.extract_data_loop(n_max)
        self.data_corrected = self.reduce_df(self.data_corrected)



        

    def word_completion(self):
        
        self.word_completor.df, self.word_completor.page_id_colname, self.word_completor.text_colname, self.word_completor.output_filename = self.data_corrected, "page_id", "text", None
        n_max= len(self.data_corrected)
        self.word_completor.prompt = fw_config["COMPLETION_PROMPT"]
        self.word_completor.model_name = fw_config["model_name"]
        data_word_completed = self.word_completor.extract_data_loop(n_max)
        data_word_completed = self.reduce_df(data_word_completed)
        return data_word_completed

        
                


    def flow(self):
        self.text_extraction()
        self.text_correction()
        return self.word_completion()



In [6]:
test = FullWorkflow(df.page_id[0:10].tolist(), df.plainpagefulltext[0:10].tolist())
result = test.flow()


Error while parsing the response - Exeption Unterminated string starting at: line 445 column 5 (char 221745)
Error while parsing the response - Exeption Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google.dev/api/generate-content#finishreason) is 8.


In [9]:
result

,page_id,text,time
0,ZXFS2473UNWC3CHAEZ72NJHVO25D2MS4-ALTO10099161_...,"Heiratsgesuch. Ein Witwer, katholisch, 40 Jahr...",2025-09-27_14_35
1,ZXFS2473UNWC3CHAEZ72NJHVO25D2MS4-ALTO10099161_...,"Zeit. Kaufmann, hier fremd, 42 Jahre, angenehm...",2025-09-27_14_35
2,ZXU6KYKLG5L4S5DGG3MPQASQD5V2ML3P-ALTO7642365_D...,"In kinderloses Ehepaar katholisch, wünscht ein...",2025-09-27_14_35
3,ZY2Z6ZIEIEB3Y3TYZ5XPUWUGP4A7T7XQ-ALTO8681078_D...,"Die Erstellung des DDB-Portals, das eine zentr...",2025-09-27_14_35
4,ZY5B654V3GOD5BG53AUUT7RZWZIAGER6-ALTO7678198_D...,"un homme , porteur d'un diplôme secondaire sui...",2025-09-27_14_35
5,ZYVLIQOXXPQPCF6IA25NUDPGDKTOMSTY-ALTO8702353_D...,"Heiratsgesuch. Malermeister, achtundzwanzig, k...",2025-09-27_14_35
6,ZZ4LWGXO7L2TEVDZ5F42M46W32X4MX2Q-ALTO8250057_D...,"Ich suche für meinen Freund, aktiven Hauptmann...",2025-09-27_14_35
7,ZZ4LWGXO7L2TEVDZ5F42M46W32X4MX2Q-ALTO8250057_D...,"Reelles Heiratsgesuch. Junger Mann, 30 Jahre a...",2025-09-27_14_35
8,ZZ4LWGXO7L2TEVDZ5F42M46W32X4MX2Q-ALTO8250057_D...,"Staatsbeamter mit hohem Gehalt, evangelisch, 3...",2025-09-27_14_35
9,ZZ4LWGXO7L2TEVDZ5F42M46W32X4MX2Q-ALTO8250057_D...,"Heirat. Suche Verwandte, jugendliche schöne Er...",2025-09-27_14_35
